In [2]:
import pandas as pd

In [6]:
pd.set_option("max_colwidth", 0)

# What do we need for our analysis

## What do we want to find out

1. Is the [recommended strategy](http://www.lasvegas-how-to.com/blackjack.php) for blackjack better than just playing randomly?
    - Does the number of decks of cards affect our results?
        - 1, 4, 6 or 8
    - Does the number of times we hit affect our results?
3. Can we do better than the [recommended strategy](http://www.lasvegas-how-to.com/blackjack.php)?

## What data do we need

- Game data
- For both random and recommended strategies
- From the player's perspective
    - We only see one of the dealer's cards
    - We don't care what cards we have, only how many points we have
- Possible outcomes
    - We win
    - We lose
    - We draw with the dealer
- Actions
    - Does the dealer need to hit
    - Does the player need to hit
    - Do we care about how many times?

# Analyzing the game

## How does a game end

- Player gets blackjack
- Dealer gets blackjack
- Player hits and busts
- Player finishes hitting, dealer hits and busts
- Player and dealer finish hitting

## Outcomes

- player gets blackjack, dealer didn't
    - player wins
- dealer gets blackjack, player didn't
    - player loses
- player hits and busts
    - player loses
- player hits, dealer hits and busts
    - player wins
- player hits, dealer hits/ dealer >= hard 17
    - player > dealer
        - player wins
    - player < dealer
        - player loses
    - player == dealer
        - draw

## Calculating points

- J, Q, K == 10
- A == 11 if it doesn't cause the hand to bust, otherwise A == 1

## [Recommended strategy](http://www.lasvegas-how-to.com/blackjack.php)

- Stand on a hand of 17 or more 
- Never hit on 12, 13, 14, 15 or 16 when the dealer is showing 16 or less
- Always split 8's
- Double down on 11 if dealer is showing 17 or less

- If your hand is 11 or less, always hit. You can't possibly bust, so the extra card will only help your hand.
- If your hand is 17 or greater, and the dealer is not showing a Seven or higher, always stand. The risk of busting if you hit is very high.
- If your hand is 12 to 16, and the dealer is showing an up card likely to bust , always stand. Although this is a weak hand, it will still win if the dealer busts.
    - Exception - If the dealer has a Two or a Three showing, hit if you have 12.
- If your hand is 12 to 16, and the dealer has a Seven or higher showing, always hit. 
    - The dealer is far more likely to get a better hand than yours unless you can improve it. 
    - There's a risk of busting, but in this case you have to take that risk.

# Coding the game

## Assumptions

- only one player (for simplicity)
- continuous shuffling machines used
    - it's as though we're using a fresh deck each time
- player always goes first

## Requirements

- deck of cards
    - suits not required
    - multiple decks
    - need to be able to shuffle the deck
    - need to draw a card from the deck
- points calculator
- game simulator
    - does the player hit
    - does the dealer hit

In [10]:
data_dictionary = pd.DataFrame([["dealer_open", "Int", "The card we can see"], ["dealer_initial", "Int", "The dealer's starting points"], 
                                ["dealer_hit", "Binary", "1 - dealer hit, 0 - dealer did not hit"], 
                                ["dealer_num_hits", "Int", "Number of times the dealer hit"], 
                                ["dealer_final", "Int", "The dealer's final points"], 
                                ["dealer_hand", "Obj", "List of the dealer's cards, in the order that they were dealt"], 
                                ["player_inital", "Int", "The player's starting points"], 
                                ["player_hit", "Binary", "1 - player hit, 0 - player did not hit"], 
                                ["player_num_hits", "Int", "Number of times the player hit"], 
                                ["player_final", "Int", "The player's final points"], 
                                ["player_hand", "Obj", "List of the player's cards, in the order that they were dealt"], 
                                ["player_loses", "Binary", "1 - player lost, 0 - player did not lose"], 
                                ["draw", "Binary", "1 - draw, 0 - not a draw"], 
                                ["player_wins", "Binary", "1 - player won, 0 - player did not win"], 
                                ["strategy", "Binary", "1 - recommended, 0 - random"]], columns=["Feature", "Type", "Description"])

In [11]:
data_dictionary

,Feature,Type,Description
0,dealer_open,Int,The card we can see
1,dealer_initial,Int,The dealer's starting points
2,dealer_hit,Binary,"1 - dealer hit, 0 - dealer did not hit"
3,dealer_num_hits,Int,Number of times the dealer hit
4,dealer_final,Int,The dealer's final points
5,dealer_hand,Obj,"List of the dealer's cards, in the order that they were dealt"
6,player_inital,Int,The player's starting points
7,player_hit,Binary,"1 - player hit, 0 - player did not hit"
8,player_num_hits,Int,Number of times the player hit
9,player_final,Int,The player's final points


In [12]:
data_dictionary.to_csv("data_dictionary.csv", index=False)